**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Prepare

**Set global variable**

In [2]:
TXT_FNAME_INP = "region.annotation.fcc_starrmpra_junke.assayvote.tsv"
TXT_FNAME_OUT = "region.annotation.genome_tss.tsv"

**View files**

In [3]:
### get annotated region folder
txt_fdiry = file.path(FD_RES, "region_annotation")
vec = dir(txt_fdiry)
names(vec) = vec

### assign and show
vec_txt_folder = vec
for(txt in vec){cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


In [4]:
txt_fdiry = file.path(FD_RES, "region_annotation", "*", "summary")
txt_fname = TXT_FNAME_INP
txt_fglob = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for(txt in vec){cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_overlap/summary/region.annotation.fcc_starrmpra_junke.assayvote.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_union/summary/region.annotation.fcc_starrmpra_junke.assayvote.tsv 


## Import data

In [6]:
lst = lapply(vec_txt_folder, function(txt_folder){
    ### set file directory
    txt_fdiry = file.path(
        FD_RES, 
        "region_annotation", 
        txt_folder,
        "summary"
    )
    txt_fname = "region.annotation.fcc_starrmpracrispr.concat.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### read table
    dat = read_tsv(txt_fpath, show_col_types = FALSE)
    return(dat)
})
names(lst) = vec_txt_folder

### assign and show
lst_dat_region_annot_fcc_peak_merge = lst
res = lapply(lst, dim)
print(res)
dat = lst[[1]]
fun_display_table(head(dat, 3))

$fcc_astarr_macs_input_overlap
[1] 156686      6

$fcc_astarr_macs_input_union
[1] 201420      6



Chrom,ChromStart,ChromEnd,Region,Group,Label
chr1,10038,10405,chr1:10038-10405,ASTARR,ASTARR_R
chr1,16025,16338,chr1:16025-16338,ASTARR,ASTARR_R
chr1,17288,17689,chr1:17288-17689,ASTARR,ASTARR_R


In [7]:
lst = lst_dat_region_annot_fcc_peak_merge
lst = lapply(lst, function(dat){

    dat = dat %>% 
        dplyr::select(Chrom:Region, Group) %>% 
        dplyr::distinct()
    dat = dat %>% 
        dplyr::group_by(Chrom, ChromStart, ChromEnd, Region) %>% 
        dplyr::summarise(
            Num_Assay = n(),
            Assays    = paste(Group, collapse = ","),
            .groups   = "drop"
        )
    return(dat)
})
lst_dat_region_fcc_assayvote = lst
res = lapply(lst, dim)
print(res)
dat = lst[[1]]
fun_display_table(head(dat, 3))

$fcc_astarr_macs_input_overlap
[1] 100454      6

$fcc_astarr_macs_input_union
[1] 135780      6



Chrom,ChromStart,ChromEnd,Region,Num_Assay,Assays
chr1,10038,10405,chr1:10038-10405,1,ASTARR
chr1,16025,16338,chr1:16025-16338,1,ASTARR
chr1,17288,17689,chr1:17288-17689,1,ASTARR


**Explore: Assays**

In [8]:
lst = lst_dat_region_fcc_assayvote
dat = lst[[1]]
table(dat$Assays)


                                                              ASTARR 
                                                               58744 
                                               ASTARR,CRISPRi-Growth 
                                                                1277 
                                 ASTARR,CRISPRi-Growth,E2G-Benchmark 
                                                                   3 
                                                ASTARR,CRISPRi-HCRFF 
                                                                   8 
                                 ASTARR,CRISPRi-HCRFF,CRISPRi-Growth 
                                                                   1 
                                                ASTARR,E2G-Benchmark 
                                                                  69 
                                                        ASTARR,LMPRA 
                                                                6390 
                   

**Explore: Count**

In [9]:
lst = lst_dat_region_fcc_assayvote
lst = lapply(lst, function(dat){
    dat = dat %>% dplyr::mutate(Note = paste0("N", Num_Assay))
    res = table(dat$Note, dnn=c("Note"))
    dat = as.data.frame(res)
    return(dat)
})

dat = bind_rows(lst, .id = "Region")
dat = dat %>% tidyr::spread(Note, Freq) #%>% dplyr::mutate(Total = sum(N1:N7))
fun_display_table(dat)

Region,N1,N2,N3,N4,N5,N6,N7
fcc_astarr_macs_input_overlap,66501,22994,9740,1177,31,6,5
fcc_astarr_macs_input_union,95992,28122,10384,1237,34,6,5


```
    1     2     3     4     5     6     7 
66501 22994  9740  1177    31     6     5 
```

## Save results

In [9]:
lst = lst_dat_region_fcc_assayvote
for (idx in names(lst)){

    ### get each table
    dat_region_annot_result = lst_dat_region_fcc_assayvote[[idx]]
    
    
    ### set file directory
    txt_folder = idx
    txt_fdiry = file.path(
        FD_RES, 
        "region_annotation", 
        txt_folder,
        "summary"
    )
    
    ### set file path
    txt_fname = "region.annotation.fcc_starrmpracrispr.assayvote.tsv"
    #txt_fname = "region.summary.fcc_starrmpra_junke.assayvote.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### write table
    dat = dat_region_annot_result
    write_tsv(dat, txt_fpath)
}

In [13]:
dat = lst_dat_region_fcc_assayvote[["fcc_astarr_macs_input_overlap"]]
dat = dat %>% dplyr::filter(Num_Assay > 1)
#dat = dat %>% dplyr::select(Chrom, ChromStart, ChromEnd, Region, Group)
dat = dat %>% dplyr::arrange(Chrom, ChromStart, ChromEnd) %>% dplyr::distinct()

print(dim(dat))
head(dat)

[1] 33953     6


Chrom,ChromStart,ChromEnd,Region,Num_Assay,Assays
<chr>,<dbl>,<dbl>,<chr>,<int>,<chr>
chr1,605104,605675,chr1:605104-605675,2,"ASTARR,CRISPRi-Growth"
chr1,778233,779389,chr1:778233-779389,3,"ASTARR,LMPRA,WSTARR"
chr1,818708,819335,chr1:818708-819335,2,"ASTARR,WSTARR"
chr1,826796,828040,chr1:826796-828040,3,"ASTARR,WSTARR,CRISPRi-Growth"
chr1,842516,843431,chr1:842516-843431,2,"ASTARR,LMPRA"
chr1,903783,905204,chr1:903783-905204,2,"ASTARR,LMPRA"


In [14]:
txt_fdiry = file.path(
    FD_REF, 
    "fcc_peak_call_cres"
)
txt_fname = "region.annotation.fcc_starrmpracrispr.assayvote.filtered.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

write_tsv(dat, txt_fpath)